In [1]:
#Imports
import numpy as np
from pandas.io.parsers import read_csv
import matplotlib.pyplot as plt
from matplotlib.axes import Axes
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import math
import scipy.optimize as opt
from sklearn.preprocessing import PolynomialFeatures
from scipy.io import loadmat
from scipy.optimize import minimize

import displayData as disp
import checkNNGradients as grad

In [2]:
def func_coste_reg(Thetas, X, Y, lmb):
    m = np.shape(X)[0]
    return cost_funct(Y, X, m) + regularizacion(Thetas, lmb, m)

def sigmoid_funct(z):
    return 1.0 / (1.0 + np.exp(-z))

def cost_funct(Y, g, m):
    J = np.sum(-1.0*Y* np.log(g) -1.0*(1 - Y)* np.log(1-g))
    return J/m

def regularizacion(Thetas, lmb, m):
    suma = 0
    for i in Thetas:
        i = i[:,1:]
        suma += (np.sum(i**2))
    
    return (lmb/(2*m))*suma


In [3]:
def forward_propagation(X, theta1, theta2, m, Y):
    
    #Capa entrada asignamos la X con los unos incluidos
    a1 = np.hstack([np.ones([m, 1]), X])

    #capa intermedia (hidden) calculamos las ecuaciones de la anterior, aplicamos la sigmoide e incluimos los unos de la neurona 0 
    z2 = np.dot(a1, theta1.T)
    a2 = np.hstack([np.ones([m, 1]), sigmoid_funct(z2)])

    #Capa salida calcuamos las ecuaciones con theta2 y aplicamos la sigmoide, nos devuelve la matriz de salida 5000x10
    z3 = np.dot(a2, theta2.T)
    
    a3 = sigmoid_funct(z3)
    
    return a1, z2, a2, z3, a3

In [4]:
#Genera los pesos aleatorios
def RandomWeights(entradas, salidas, ini):
    Theta = np.random.uniform(-ini, ini, size = (entradas+1, salidas))
    return Theta

In [5]:

def backprop(params_rn, num_entradas, num_ocultas, num_etiquetas, X, Y, reg):

    #Recostruimos las Thetas
    Theta1 = np.reshape(params_rn[:num_ocultas *(num_entradas + 1)],(num_ocultas, (num_entradas+1)))
    Theta2 = np.reshape(params_rn[num_ocultas*(num_entradas + 1): ], (num_etiquetas,(num_ocultas+1)))
    
    m = X.shape[0]    
    y_onehot = Y
    
    a1, z2, a2, z3, a3 = forward_propagation(X, Theta1, Theta2, m, y_onehot)
    
    #Calculamos el coste
    coste = func_coste_reg([Theta1, Theta2], a3, y_onehot, reg)

    d3 = a3 - y_onehot
    d2 = np.matmul(Theta2.T,d3.T)*(a2*(1-a2)).T
    
    #Calculamos los gradientes no regularizados
    G1 = np.matmul(d2[1:,:], a1)/m
    G2 = np.matmul(d3.T,a2)/m
    
    #Calculamos los gradientes regularizados
    G1 = G1 + ((reg*1.0)/m)*np.insert(Theta1[:, 1:], 0, 0, axis = 1)
    G2 = G2 + ((reg*1.0)/m)*np.insert(Theta2[:, 1:], 0, 0, axis = 1)

    #Ponemos los gradientes en forma de lista
    gradientes = np.concatenate((G1, G2), axis = None)
    
    return coste, gradientes

In [6]:
def optimize_backprop_and_check (RAN1, RAN2, num_entradas, num_ocultas, num_etiquetas, reg, X, y_onehot, laps, Y, ini):
    
    #Inicializamos los pesos y los ponemos en forma de lista
    #Theta1 = RandomWeights(num_entradas, num_ocultas, ini)
    #Theta2 = RandomWeights(num_ocultas, num_etiquetas, ini)
    Theta1 = RAN1;
    Theta2 = RAN2;
    pesos = np.concatenate((Theta1, Theta2), axis=None)
    
    #Optimizamos
    out = opt.minimize(fun = backprop, x0 = pesos, args = (num_entradas, num_ocultas, num_etiquetas, X, y_onehot, reg), method='TNC', jac = True, options = {'maxiter': laps})

    #Reconstruimos las Thetas
    Theta1 = out.x[:(num_ocultas*(num_entradas+1))].reshape(num_ocultas,(num_entradas+1))
    Theta2 = out.x[(num_ocultas*(num_entradas+1)):].reshape(num_etiquetas,(num_ocultas+1))
    
    m = X.shape[0]
    #Hacemos el forward propagation
    a1, z2, a2, z3, a3 = forward_propagation(X, Theta1, Theta2, m, y_onehot)
    
    #Sacamos los aciertos
    indexes = np.argmax(a3, axis=1)
    acc = (np.sum(indexes == (Y))/m)*100
    
    #print(acc)
    return acc

In [7]:
def main():
    
    data = loadmat ("ex4data1.mat")

    #almacenamos los datos leídos en X e y
    X = data['X']
    m = X.shape[0]
    #X = np.hstack([np.ones([m, 1]), X])
    y = data['y']
    y = np.ravel(y) 
    num_labels = 10
    num_entries = np.shape(X)[1]
    num_hiden_layers = 25

    #print(X.shape)
    #disp.displayImage(X[4])
    #disp.displayImage(X[4000])
    #disp.displayImage(X[500])
    #disp.displayImage(X[1000])
    
    #Ponemos la y en forma onehot
    y = y-1
    y_onehot = np.zeros((m, num_labels))  # 5000 x 10
    
    for i in range(m):
        y_onehot[i][y[i]] = 1
    
    weights = loadmat("ex4weights.mat")
    theta1, theta2 = weights["Theta1"], weights["Theta2"]
    reg = 1
    a1, z2, a2, z3, a3 = forward_propagation(X, theta1, theta2, m, y_onehot)
    
    #print("Coste no regulado: ",cost_funct(y_onehot, a3, m))
    #print("Coste regulado: ",func_coste_reg([theta1, theta2], a3,y_onehot, reg))
    
    #Check del gradiente con reg
    #print(grad.checkNNGradients(backprop, reg))
    num_labels = 10
    num_entries = np.shape(X)[1]
    num_hiden_layers = 25

    ini = 0.12
    RAN11 = np.random.uniform(-ini, ini, size = (num_entries+1, num_hiden_layers))
    RAN21 = np.random.uniform(-ini, ini, size = (num_hiden_layers+1, num_labels))
    RAN12 = np.random.uniform(-ini, ini, size = (num_entries+1, num_hiden_layers))
    RAN22 = np.random.uniform(-ini, ini, size = (num_hiden_layers+1, num_labels))
    RAN13 = np.random.uniform(-ini, ini, size = (num_entries+1, num_hiden_layers))
    RAN23 = np.random.uniform(-ini, ini, size = (num_hiden_layers+1, num_labels))

    lambdas = [0, 0.1, 0.25, 0.5, 1, 2, 3, 4, 5, 10, 20, 50, 100]
    for i in lambdas:
        print("lambda =", i)
        acc = optimize_backprop_and_check(RAN11, RAN21, num_entries, num_hiden_layers, num_labels, i, X, y_onehot, 70, y, 0.12)
        print("aciertos:", acc, "%")
        acc = optimize_backprop_and_check(RAN12, RAN22, num_entries, num_hiden_layers, num_labels, i, X, y_onehot, 70, y, 0.12)
        print("aciertos:", acc, "%")
        acc = optimize_backprop_and_check(RAN13, RAN23, num_entries, num_hiden_layers, num_labels, i, X, y_onehot, 70, y, 0.12)
        print("aciertos:", acc, "%")
    print("\n")

    #return acc

In [8]:
main()

lambda = 0
aciertos: 93.24 %
aciertos: 89.02 %
aciertos: 91.16 %
lambda = 0.1
aciertos: 93.14 %
aciertos: 92.4 %
aciertos: 90.9 %
lambda = 0.25
aciertos: 90.75999999999999 %
aciertos: 92.80000000000001 %
aciertos: 94.6 %
lambda = 0.5
aciertos: 93.30000000000001 %
aciertos: 91.02 %
aciertos: 92.30000000000001 %
lambda = 1
aciertos: 91.44 %
aciertos: 92.58 %
aciertos: 90.60000000000001 %
lambda = 2
aciertos: 93.97999999999999 %
aciertos: 91.97999999999999 %
aciertos: 93.10000000000001 %
lambda = 3
aciertos: 92.44 %
aciertos: 90.24 %
aciertos: 91.46 %
lambda = 4
aciertos: 93.42 %
aciertos: 91.82000000000001 %
aciertos: 91.66 %
lambda = 5
aciertos: 83.48 %
aciertos: 91.67999999999999 %
aciertos: 91.84 %
lambda = 10
aciertos: 90.08 %
aciertos: 92.62 %
aciertos: 92.44 %
lambda = 20
aciertos: 87.9 %
aciertos: 90.2 %
aciertos: 89.75999999999999 %
lambda = 50
aciertos: 88.74 %
aciertos: 88.92 %
aciertos: 84.39999999999999 %
lambda = 100
aciertos: 48.42 %
aciertos: 84.94 %
aciertos: 83.3 %




In [9]:
num_iter = 10
acc = 0
for i in range(0, num_iter):
    main()
    #acc = acc + main()
#acc = acc/num_iter
#print(acc, "\n\n")

lambda = 0
aciertos: 86.88 %
aciertos: 93.14 %
aciertos: 94.22 %
lambda = 0.1
aciertos: 93.30000000000001 %
aciertos: 92.97999999999999 %
aciertos: 91.75999999999999 %
lambda = 0.25
aciertos: 93.34 %
aciertos: 89.58 %
aciertos: 94.14 %
lambda = 0.5
aciertos: 92.82000000000001 %
aciertos: 93.56 %
aciertos: 93.04 %
lambda = 1
aciertos: 94.24 %
aciertos: 93.16 %
aciertos: 93.54 %
lambda = 2
aciertos: 95.12 %
aciertos: 93.36 %
aciertos: 92.9 %
lambda = 3
aciertos: 91.74 %
aciertos: 93.10000000000001 %
aciertos: 91.78 %
lambda = 4
aciertos: 93.97999999999999 %
aciertos: 93.36 %
aciertos: 92.02 %
lambda = 5
aciertos: 92.7 %
aciertos: 92.66 %
aciertos: 90.2 %
lambda = 10
aciertos: 91.52 %
aciertos: 91.16 %
aciertos: 90.25999999999999 %
lambda = 20
aciertos: 91.4 %
aciertos: 89.9 %
aciertos: 89.5 %
lambda = 50
aciertos: 86.58 %
aciertos: 88.32 %
aciertos: 88.62 %
lambda = 100
aciertos: 81.66 %
aciertos: 82.84 %
aciertos: 84.17999999999999 %


lambda = 0
aciertos: 93.38 %
aciertos: 94.44 %
acie

aciertos: 89.8 %
aciertos: 90.03999999999999 %
aciertos: 89.52 %
lambda = 50
aciertos: 88.84 %
aciertos: 88.28 %
aciertos: 88.44 %
lambda = 100
aciertos: 80.80000000000001 %
aciertos: 78.74 %
aciertos: 83.5 %


lambda = 0
aciertos: 92.92 %
aciertos: 94.17999999999999 %
aciertos: 90.86 %
lambda = 0.1
aciertos: 91.22 %
aciertos: 93.67999999999999 %
aciertos: 90.74 %
lambda = 0.25
aciertos: 93.66 %
aciertos: 93.78 %
aciertos: 91.46 %
lambda = 0.5
aciertos: 92.56 %
aciertos: 90.16 %
aciertos: 91.16 %
lambda = 1
aciertos: 92.4 %
aciertos: 92.92 %
aciertos: 91.32000000000001 %
lambda = 2
aciertos: 92.46 %
aciertos: 92.66 %
aciertos: 92.0 %
lambda = 3
aciertos: 93.34 %
aciertos: 93.02 %
aciertos: 91.96 %
lambda = 4
aciertos: 91.53999999999999 %
aciertos: 90.94 %
aciertos: 90.96 %
lambda = 5
aciertos: 92.64 %
aciertos: 79.4 %
aciertos: 91.7 %
lambda = 10
aciertos: 89.52 %
aciertos: 91.92 %
aciertos: 90.64 %
lambda = 20
aciertos: 87.76 %
aciertos: 90.84 %
aciertos: 89.56 %
lambda = 50
aciertos:

In [10]:
num_iter = 250
